# InitialSetup

In [1]:
import os
import shutil

if not os.path.exists('data_set/'):
    raise Exception("Generated dataset not found, pls run 'SetupDataset' first.")
    
if not os.path.exists('OUTPUT/'):
    os.makedirs('OUTPUT/')

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, AveragePooling2D, MaxPooling2D, Convolution2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

2023-09-05 15:06:28.225011: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Loading dataset

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255, brightness_range=[0.2,1.0], shear_range=0.2,
                                    fill_mode = 'nearest', width_shift_range=0.2, rotation_range=40,
                                   height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2)
training_data = train_gen.flow_from_directory('data_set/train', target_size=(256,256), 
                                                 batch_size=32, class_mode='binary')

val_gen = ImageDataGenerator(rescale=1./255)
val_data = val_gen.flow_from_directory('data_set/val', target_size=(256, 256), 
                                            batch_size=32, class_mode='binary')

test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory('data_set/test', target_size=(256, 256), 
                                            batch_size=32, class_mode='binary')

Found 2489 images belonging to 2 classes.
Found 533 images belonging to 2 classes.
Found 536 images belonging to 2 classes.


# Training model from scratch

In [4]:
classifier = Sequential([
    Convolution2D(64,3,3, input_shape=(256,256,3), activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Convolution2D(32,3,3, activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Flatten(),
    
    Dense(256, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [5]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [6]:
steps_per_epoch = len(training_data)
validation_steps = len(val_data)

classifier.fit(training_data, epochs=1, steps_per_epoch=steps_per_epoch, workers=1,
                    validation_data=val_data, validation_steps=validation_steps)

2023-09-05 15:06:31.967417: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


78/78 [==============================] - ETA: 0s - loss: 0.5137 - accuracy: 0.7336

2023-09-05 15:08:15.456270: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


78/78 [==============================] - 112s 1s/step - loss: 0.5137 - accuracy: 0.7336 - val_loss: 0.2188 - val_accuracy: 0.9137


In [7]:
classifier.save("OUTPUT/ScratchModel.h5")

In [8]:
_, accuracy = classifier.evaluate(test_data, steps = len(test_data))

2023-09-05 15:08:24.125141: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


17/17 [==============================] - 6s 338ms/step - loss: 0.2123 - accuracy: 0.9086


In [9]:
print('ScratchModel accuracy :',accuracy)

ScratchModel accuracy : 0.9085820913314819
